In [1]:
%load_ext autoreload
%autoreload 2

# Tinygrid ERCOT API Demo

This notebook demonstrates how to use tinygrid to access ERCOT grid data with a clean, unified API.

**Features:**
- Type-safe enums for markets and location types
- Convenient date handling with keywords like "today" and "yesterday"
- Unified methods that route to the correct endpoints automatically
- Built-in location filtering

## Setup

First, set your ERCOT API credentials as environment variables:

```bash
export ERCOT_USERNAME="your-email@example.com"
export ERCOT_PASSWORD="your-password"
export ERCOT_SUBSCRIPTION_KEY="your-subscription-key"
```

Or create a `.env` file in the examples directory.

In [2]:
import os

import pandas as pd
from dotenv import load_dotenv

from tinygrid import ERCOT, ERCOTAuth, ERCOTAuthConfig, LocationType, Market

# Load environment variables from .env file
load_dotenv()
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 200)

## Create Authenticated Client

In [3]:
# Create authenticated client
auth = ERCOTAuth(ERCOTAuthConfig(
    username=os.environ["ERCOT_USERNAME"],
    password=os.environ["ERCOT_PASSWORD"],
    subscription_key=os.environ["ERCOT_SUBSCRIPTION_KEY"],
))

ercot = ERCOT(auth=auth)
print("Client created successfully!")

Client created successfully!


## Market and Location Types

Tinygrid uses enums for type safety and IDE autocomplete.

In [4]:
print("Available Markets:")
for m in Market:
    print(f"  Market.{m.name}")

print("\nAvailable Location Types:")
for lt in LocationType:
    print(f"  LocationType.{lt.name}")

Available Markets:
  Market.REAL_TIME_SCED
  Market.REAL_TIME_15_MIN
  Market.DAY_AHEAD_HOURLY

Available Location Types:
  LocationType.LOAD_ZONE
  LocationType.TRADING_HUB
  LocationType.RESOURCE_NODE
  LocationType.ELECTRICAL_BUS


## Settlement Point Prices (SPP)

Get real-time or day-ahead settlement point prices with optional filtering.

In [12]:
# Real-time 15-minute SPP
df = ercot.get_spp(
    start="2024-12-12",
    market=Market.REAL_TIME_15_MIN,
)

print(f"Real-Time SPP: {len(df):,} records")
df.head()

Real-Time SPP: 91,390 records


,Time,End Time,Location,Price,Market,Location Type
0,2024-12-12 23:30:00-06:00,2024-12-12 23:45:00-06:00,7RNCHSLR_ALL,18.81,REAL_TIME_15_MIN,RN
1,2024-12-12 23:30:00-06:00,2024-12-12 23:45:00-06:00,ADL_RN,20.58,REAL_TIME_15_MIN,RN
2,2024-12-12 23:30:00-06:00,2024-12-12 23:45:00-06:00,AEEC,8.81,REAL_TIME_15_MIN,RN
3,2024-12-12 23:30:00-06:00,2024-12-12 23:45:00-06:00,AE_RN,18.83,REAL_TIME_15_MIN,RN
4,2024-12-12 23:30:00-06:00,2024-12-12 23:45:00-06:00,AGUAYO_UNIT1,8.88,REAL_TIME_15_MIN,RN


In [23]:
# Filter to Load Zones only
df = ercot.get_spp(
    start="yesterday",
    market=Market.REAL_TIME_15_MIN,
    location_type=LocationType.LOAD_ZONE,
)

print(f"Load Zone SPP: {len(df):,} records")
df.head()

Load Zone SPP: 1,520 records


,Time,End Time,Location,Price,Market,Location Type
0,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_AEN,8.10,REAL_TIME_15_MIN,LZ
1,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_AEN,8.10,REAL_TIME_15_MIN,LZEW
2,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_CPS,7.73,REAL_TIME_15_MIN,LZEW
3,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_CPS,7.73,REAL_TIME_15_MIN,LZ
4,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_HOUSTON,9.89,REAL_TIME_15_MIN,LZEW


In [22]:
# Filter to specific locations
df = ercot.get_spp(
    start="yesterday",
    market=Market.REAL_TIME_15_MIN,
    locations=["LZ_HOUSTON", "LZ_NORTH", "HB_HOUSTON"],
)

print(f"Specific locations: {len(df):,} records")
df

Specific locations: 475 records


,Time,End Time,Location,Price,Market,Location Type
0,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,HB_HOUSTON,9.96,REAL_TIME_15_MIN,HU
1,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_HOUSTON,9.89,REAL_TIME_15_MIN,LZEW
2,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_HOUSTON,9.89,REAL_TIME_15_MIN,LZ
3,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_NORTH,9.87,REAL_TIME_15_MIN,LZ
4,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_NORTH,9.88,REAL_TIME_15_MIN,LZEW
...,...,...,...,...,...,...
470,2025-12-27 00:00:00-06:00,2025-12-27 00:15:00-06:00,HB_HOUSTON,13.94,REAL_TIME_15_MIN,HU
471,2025-12-27 00:00:00-06:00,2025-12-27 00:15:00-06:00,LZ_HOUSTON,13.93,REAL_TIME_15_MIN,LZ
472,2025-12-27 00:00:00-06:00,2025-12-27 00:15:00-06:00,LZ_HOUSTON,13.93,REAL_TIME_15_MIN,LZEW
473,2025-12-27 00:00:00-06:00,2025-12-27 00:15:00-06:00,LZ_NORTH,13.97,REAL_TIME_15_MIN,LZEW


In [25]:
# Day-Ahead SPP
df = ercot.get_spp(
    start="2025-12-28",
    market=Market.DAY_AHEAD_HOURLY,
    location_type=[LocationType.LOAD_ZONE, LocationType.TRADING_HUB],
)

print(f"Day-Ahead SPP: {len(df):,} records")
df

Day-Ahead SPP: 360 records


,Time,End Time,Location,Price,Market
0,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,HB_BUSAVG,12.61,DAY_AHEAD_HOURLY
1,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,HB_HOUSTON,12.65,DAY_AHEAD_HOURLY
2,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,HB_HUBAVG,10.58,DAY_AHEAD_HOURLY
3,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,HB_NORTH,15.98,DAY_AHEAD_HOURLY
4,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,HB_PAN,-2.69,DAY_AHEAD_HOURLY
...,...,...,...,...,...
355,2025-12-28 23:00:00-06:00,2025-12-29 00:00:00-06:00,LZ_LCRA,13.70,DAY_AHEAD_HOURLY
356,2025-12-28 23:00:00-06:00,2025-12-29 00:00:00-06:00,LZ_NORTH,15.12,DAY_AHEAD_HOURLY
357,2025-12-28 23:00:00-06:00,2025-12-29 00:00:00-06:00,LZ_RAYBN,18.46,DAY_AHEAD_HOURLY
358,2025-12-28 23:00:00-06:00,2025-12-29 00:00:00-06:00,LZ_SOUTH,14.25,DAY_AHEAD_HOURLY


## Locational Marginal Prices (LMP)

Get LMP data by settlement point or electrical bus.

In [10]:
# Real-time LMP by settlement point
df = ercot.get_lmp(
    start="today",
    market=Market.REAL_TIME_SCED,
)

print(f"Real-Time LMP: {len(df):,} records")
df.head()

Real-Time LMP: 1,000 records


,Location,Price,Market,SCED Time Stamp,Repeat Hour Flag
0,INGLCOS_CTG2,5.89,REAL_TIME_SCED,2025-12-28T10:30:15,False
1,INGLCO_PUN1,5.89,REAL_TIME_SCED,2025-12-28T10:30:15,False
2,INGLCO_PUN2,5.89,REAL_TIME_SCED,2025-12-28T10:30:15,False
3,INGLCO_PUN3,5.89,REAL_TIME_SCED,2025-12-28T10:30:15,False
4,INKS_INKS_G1,6.41,REAL_TIME_SCED,2025-12-28T10:30:15,False


In [11]:
# LMP by electrical bus (more granular)
df = ercot.get_lmp(
    start="today",
    market=Market.REAL_TIME_SCED,
    location_type=LocationType.ELECTRICAL_BUS,
)

print(f"Electrical Bus LMP: {len(df):,} records")
df.head()

Electrical Bus LMP: 1,000 records


,Price,Market,SCED Time Stamp,Repeat Hour Flag,Electrical Bus
0,6.41,REAL_TIME_SCED,2025-12-28T10:30:15,False,0001
1,6.50,REAL_TIME_SCED,2025-12-28T10:30:15,False,0001DUPV1_
2,2.45,REAL_TIME_SCED,2025-12-28T10:30:15,False,0001HWFG1
3,6.49,REAL_TIME_SCED,2025-12-28T10:30:15,False,0001VICTOR
4,6.59,REAL_TIME_SCED,2025-12-28T10:30:15,False,0001_C


In [17]:
# Day-Ahead LMP
df = ercot.get_lmp(
    start="2025-12-29",
    market=Market.DAY_AHEAD_HOURLY,
)

print(f"Day-Ahead LMP: {len(df):,} records")
df.head()

Day-Ahead LMP: 0 records


,Delivery Date,Hour Ending,Bus Name,LMP,DST Flag


## Ancillary Services

In [13]:
# Ancillary Service Prices (MCPC)
df = ercot.get_as_prices(start="yesterday")

print(f"AS Prices: {len(df):,} records")
df.head()

AS Prices: 120 records


,Date,Hour Ending,Ancillary Type,MCPC,DST
120,2025-12-26,01:00,REGDN,0.52,False
121,2025-12-26,01:00,REGUP,0.38,False
122,2025-12-26,01:00,RRS,0.38,False
123,2025-12-26,01:00,NSPIN,3.00,False
124,2025-12-26,01:00,ECRS,0.47,False


In [14]:
# Ancillary Service Plan (requirements by hour)
df = ercot.get_as_plan(start="yesterday")

print(f"AS Plan: {len(df):,} records")
df.head()

AS Plan: 400 records


,Posted,Date,Hour Ending,Ancillary Type,Quantity,DST
240,2025-12-26T05:00:00,2025-12-26,01:00,ECRS,864,False
241,2025-12-26T05:00:00,2025-12-26,01:00,NSPIN,2278,False
242,2025-12-26T05:00:00,2025-12-26,01:00,REGDN,315,False
243,2025-12-26T05:00:00,2025-12-26,01:00,REGUP,369,False
244,2025-12-26T05:00:00,2025-12-26,01:00,RRS,2982,False


## Shadow Prices

In [15]:
# SCED Shadow Prices (real-time)
df = ercot.get_shadow_prices(
    start="yesterday",
    market=Market.REAL_TIME_SCED,
)

print(f"SCED Shadow Prices: {len(df):,} records")
df.head()

SCED Shadow Prices: 1,731 records


,SCEDTimeStamp,Repeated Hour,Constraint ID,Constraint Name,Contingency Name,Shadow Price,Max Shadow Price,Limit,Value,Violated MW,From Station,To Station,From Station kV,To Station kV,CCT Status
0,12/26/2025 22:55:14,N,1,HARGRO_TWINBU1_1,DBAKCED5,0.00000,3500,149.5,125.2,-24.3,TWINBU,HARGROVE,138,138,COMP
1,12/26/2025 22:55:14,N,2,PNHNDL,BASE CASE,17.49570,5251,2424.4,2424.4,0.0,NaN,NaN,0,0,NONCOMP
2,12/26/2025 22:55:14,N,3,NORTMC_AT2L,DBAKSOL5,0.00000,3500,874.2,788.2,-86.1,NORTMC,NORTMC,1,138,COMP
3,12/26/2025 22:50:14,N,1,HARGRO_TWINBU1_1,DBAKCED5,0.00000,3500,149.5,126.5,-23.0,TWINBU,HARGROVE,138,138,COMP
4,12/26/2025 22:50:14,N,2,PNHNDL,BASE CASE,17.64291,5251,2424.4,2424.4,0.0,NaN,NaN,0,0,NONCOMP


In [16]:
# Day-Ahead Shadow Prices
df = ercot.get_shadow_prices(
    start="yesterday",
    market=Market.DAY_AHEAD_HOURLY,
)

print(f"DAM Shadow Prices: {len(df):,} records")
df.head()

DAM Shadow Prices: 270 records


,Date,Hour Ending,Constraint Id,Constraint Name,Contingency Name,Constraint Limit,Constraint Value,Violation Amount,Shadow Price,From Station,To Station,From Station kV,To Station kV,Delivery Time,DST
730,2025-12-26,01:00,1,DEC_G1NX,BASE CASE,70,70,0,0.004,DEC,DEC,13.8,1.0,2025-12-26T01:00:00,False
731,2025-12-26,01:00,2,DEC_G1NY,BASE CASE,70,70,0,0.004,DEC,DEC,13.8,1.0,2025-12-26T01:00:00,False
732,2025-12-26,01:00,3,PNHNDL,BASE CASE,2480,2480,0,4.576,,,0.0,0.0,2025-12-26T01:00:00,False
733,2025-12-26,01:00,4,587__A,MRNKDHM5,219,219,0,429.801,ARGYL,LWSVH,138.0,138.0,2025-12-26T01:00:00,False
734,2025-12-26,01:00,5,6437__F,DMTSCOS5,221,221,0,0.382,SCRCV,KNAPP,138.0,138.0,2025-12-26T01:00:00,False


## System Load

In [19]:
# Load by weather zone
df = ercot.get_load(start="yesterday", by="weather_zone")

print(f"System Load: {len(df):,} records")
df.head()

GridAPIError: Unexpected error calling ERCOT API: sync() got an unexpected keyword argument 'oper_day_from'. Did you mean 'operating_day_from'?

## Wind & Solar Forecasts

In [18]:
# Wind forecast
df = ercot.get_wind_forecast(start="yesterday")

print(f"Wind Forecast: {len(df):,} records")
df.head()

Wind Forecast: 96 records


,Posted,Date,Hour Ending,Generation System Wide,COP HSL System Wide,STWPF System Wide,WGRPP System Wide,Generation Load Zone South Houston,COP HSL Load Zone South Houston,STWPF Load Zone South Houston,...,Generation Load Zone West,COP HSL Load Zone West,STWPF Load Zone West,WGRPP Load Zone West,Generation Load Zone North,COP HSL Load Zone North,STWPF Load Zone North,WGRPP Load Zone North,HSL System Wide,DST
167,2025-12-26T23:55:35,2025-12-26,1,21732.82,24264.0,24085.1,22994.1,2995.13,3599.0,3090.2,...,15869.55,17991.5,18130.6,17416.9,2868.14,2673.5,2864.3,2769.4,24142.55,False
168,2025-12-26T23:55:35,2025-12-26,2,20118.39,22883.6,22404.2,21297.1,2569.59,3158.2,2663.2,...,14770.90,17098.7,16964.1,16236.8,2777.90,2626.7,2776.9,2679.3,22456.45,False
169,2025-12-26T23:55:35,2025-12-26,3,18879.05,21599.8,20844.1,19739.9,2117.87,2674.4,2167.1,...,14090.81,16370.3,16024.1,15288.6,2670.37,2555.1,2652.9,2553.9,20894.14,False
170,2025-12-26T23:55:35,2025-12-26,4,17998.08,20336.8,19790.0,18684.2,1963.22,2417.5,1979.1,...,13372.45,15415.9,15167.5,14430.1,2662.41,2503.4,2643.4,2543.7,19845.25,False
171,2025-12-26T23:55:35,2025-12-26,5,17573.10,18959.7,19155.8,18043.3,1961.55,1950.6,1944.5,...,13001.00,14596.8,14621.8,13878.0,2610.55,2412.3,2589.5,2489.7,19280.22,False


In [21]:
# Solar forecast
df = ercot.get_solar_forecast(start="yesterday")

print(f"Solar Forecast: {len(df):,} records")
df.head()

Solar Forecast: 96 records


,Posted,Date,Hour Ending,Generation System Wide,COP HSL System Wide,STPPF System Wide,PVGRPP System Wide,HSL System Wide,DST
167,2025-12-26T23:55:32,2025-12-26,1,0.27,0.0,0.0,0.0,0.74,False
168,2025-12-26T23:55:32,2025-12-26,2,0.32,0.0,0.0,0.0,0.84,False
169,2025-12-26T23:55:32,2025-12-26,3,0.31,0.0,0.0,0.0,0.87,False
170,2025-12-26T23:55:32,2025-12-26,4,0.29,0.0,0.0,0.0,0.87,False
171,2025-12-26T23:55:32,2025-12-26,5,0.26,0.0,0.0,0.0,0.77,False


## Date Range Queries

Fetch data across multiple days.

In [20]:
# Get a week of data
df = ercot.get_spp(
    start="2024-12-20",
    end="2024-12-27",
    market=Market.DAY_AHEAD_HOURLY,
    location_type=LocationType.LOAD_ZONE,
)

print(f"Week of DAM SPP: {len(df):,} records")
df.head()

Week of DAM SPP: 1,152 records


,Date,Hour Ending,Location,Price,DST,Market
23348,12/26/2024,01:00,LZ_AEN,16.64,N,DAY_AHEAD_HOURLY
23349,12/26/2024,01:00,LZ_CPS,16.76,N,DAY_AHEAD_HOURLY
23350,12/26/2024,01:00,LZ_HOUSTON,16.72,N,DAY_AHEAD_HOURLY
23351,12/26/2024,01:00,LZ_LCRA,16.78,N,DAY_AHEAD_HOURLY
23352,12/26/2024,01:00,LZ_NORTH,16.75,N,DAY_AHEAD_HOURLY


## API Reference

| Method | Description | Markets |
|--------|-------------|--------|
| `get_spp()` | Settlement Point Prices | REAL_TIME_15_MIN, DAY_AHEAD_HOURLY |
| `get_lmp()` | Locational Marginal Prices | REAL_TIME_SCED, DAY_AHEAD_HOURLY |
| `get_as_prices()` | Ancillary Service MCPC | - |
| `get_as_plan()` | AS Requirements | - |
| `get_shadow_prices()` | Transmission Constraints | REAL_TIME_SCED, DAY_AHEAD_HOURLY |
| `get_load()` | System Load | - |
| `get_wind_forecast()` | Wind Generation Forecast | - |
| `get_solar_forecast()` | Solar Generation Forecast | - |